In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn

path = "D:\\2022lendingclub_ML\\"

In [2]:
data = pd.read_feather(path + "accepted_2007_to_2018Q4.feather")

##### 一、特征工程与特征筛选

In [3]:
import toad

var_glance = toad.detect(data)

In [4]:
var_glance

,type,size,missing,unique,mean_or_top1,std_or_top2,min_or_top3,1%_or_top4,10%_or_top5,50%_or_bottom5,75%_or_bottom4,90%_or_bottom3,99%_or_bottom2,max_or_bottom1
id,object,2260701,0.00%,2260701,68407277:0.00%,134840815:0.00%,134799148:0.00%,134288565:0.00%,134852755:0.00%,140937187:0.00%,140924575:0.00%,140815323:0.00%,140949487:0.00%,Total amount funded in policy code 2: 52195317...
member_id,float64,2260701,100.00%,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
loan_amnt,float64,2260701,0.00%,1572,15046.931228,9190.245488,500.0,1525.0,5000.0,12900.0,20000.0,30000.0,40000.0,40000.0
funded_amnt,float64,2260701,0.00%,1572,15041.664057,9188.413022,500.0,1516.75,5000.0,12875.0,20000.0,30000.0,40000.0,40000.0
funded_amnt_inv,float64,2260701,0.00%,10057,15023.437745,9192.331679,0.0,1500.0,5000.0,12800.0,20000.0,30000.0,40000.0,40000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
settlement_date,object,2260701,98.49%,90,Jan-2019:0.08%,Oct-2018:0.07%,Feb-2019:0.06%,Mar-2018:0.06%,Sep-2018:0.06%,Jul-2011:0.00%,Mar-2012:0.00%,Feb-2010:0.00%,Mar-2009:0.00%,Dec-2012:0.00%
settlement_amount,float64,2260701,98.49%,21941,5010.664267,3693.12259,44.21,382.0,1174.6,4146.11,6850.1725,10041.0,16691.3,33601.0
settlement_percentage,float64,2260701,98.49%,2070,47.780365,7.311822,0.2,30.02,40.02,45.0,50.0,55.07,65.071,521.35
settlement_term,float64,2260701,98.49%,40,13.191322,8.15998,0.0,0.0,1.0,14.0,18.0,24.0,24.0,181.0


##### 核心变量loan_status与因变量构建

In [5]:
var_glance.loc['loan_status',:]

type                                                         object
size                                                        2260701
missing                                                       0.00%
unique                                                            9
mean_or_top1                                      Fully Paid:47.63%
std_or_top2                                          Current:38.85%
min_or_top3                                      Charged Off:11.88%
1%_or_top4                                 Late (31-120 days):0.95%
10%_or_top5                                   In Grace Period:0.37%
50%_or_bottom5                                In Grace Period:0.37%
75%_or_bottom4                              Late (16-30 days):0.19%
90%_or_bottom3    Does not meet the credit policy. Status:Fully ...
99%_or_bottom2    Does not meet the credit policy. Status:Charge...
max_or_bottom1                                        Default:0.00%
Name: loan_status, dtype: object

In [6]:
data['loan_status'].unique()

array(['Fully Paid', 'Current', 'Charged Off', 'In Grace Period',
       'Late (31-120 days)', 'Late (16-30 days)', 'Default', None,
       'Does not meet the credit policy. Status:Fully Paid',
       'Does not meet the credit policy. Status:Charged Off'],
      dtype=object)

In [7]:
# 根据贷款的不同情况，fully paid和current是好客户，late、charged off和default是坏客户。
# grace period（宽限期）根据贷款期限的长短，可能是好客户，也可能是坏客户。对现金贷而言，宽限期也算是坏客户，但如果希望误杀率低些，不妨将其定义为好客户。这里采取好客户定义。
# 以及，这里出现的'Does not meet the credit policy. Status:X'类型贷款，可以看做是信用分低，但经人工审核后仍然发放的情况，也视作一般贷款，还了就是好客户，违约就是坏客户。

mapping_dict = {'Fully Paid':0, 'Current':0, 'In Grace Period':0, 'Does not meet the credit policy. Status:Fully Paid':0,
'Late (31-120 days)':1, 'Late (16-30 days)':1, 'Default':1, 'Charged Off':1, 'Does not meet the credit policy. Status:Charged Off':1, None: 1}


# 定义好坏客户的因变量。按此标准，坏客户比例全时期有13%左右，经验证，把宽限期算成坏客户，最终坏客户比例也不过13.4%。所以这样做区别不大。
data['target'] = data['loan_status'].apply(lambda x: mapping_dict[x])

# 另一种定义：把宽限期也算成坏客户
#mapping_dict_alter = {'Fully Paid':0, 'Current':0, 'In Grace Period':1, 'Does not meet the credit policy. Status:Fully Paid':0,
#'Late (31-120 days)':1, 'Late (16-30 days)':1, 'Default':1, 'Charged Off':1, 'Does not meet the credit policy. Status:Charged Off':1, None: 1}

#data['target'] = data['loan_status'].apply(lambda x: mapping_dict_alter[x])
#sum(data['target'])/len(data['target'])

In [8]:
var_obj = var_glance[var_glance['type']=='object']
var_obj

,type,size,missing,unique,mean_or_top1,std_or_top2,min_or_top3,1%_or_top4,10%_or_top5,50%_or_bottom5,75%_or_bottom4,90%_or_bottom3,99%_or_bottom2,max_or_bottom1
id,object,2260701,0.00%,2260701,68407277:0.00%,134840815:0.00%,134799148:0.00%,134288565:0.00%,134852755:0.00%,140937187:0.00%,140924575:0.00%,140815323:0.00%,140949487:0.00%,Total amount funded in policy code 2: 52195317...
term,object,2260701,0.00%,2,36 months:71.21%,60 months:28.79%,None,None,None,None,None,None,36 months:71.21%,60 months:28.79%
grade,object,2260701,0.00%,7,B:29.35%,C:28.75%,A:19.15%,D:14.35%,E:6.00%,A:19.15%,D:14.35%,E:6.00%,F:1.85%,G:0.54%
sub_grade,object,2260701,0.00%,35,C1:6.45%,B5:6.21%,B4:6.18%,B3:5.82%,C2:5.80%,G1:0.18%,G2:0.12%,G3:0.09%,G4:0.08%,G5:0.07%
emp_title,object,2260701,7.39%,512694,Teacher:1.72%,Manager:1.52%,Owner:0.97%,Registered Nurse:0.70%,Driver:0.65%,Electronic System Engineer:0.00%,Substitute Teacher/Paraprofessional:0.00%,eligiblity worker:0.00%,Escalated Customer Service Advisor:0.00%,Current Operations Officer:0.00%
emp_length,object,2260701,6.50%,11,10+ years:33.09%,2 years:9.01%,< 1 year:8.40%,3 years:8.00%,1 year:6.56%,4 years:6.04%,6 years:4.54%,7 years:4.10%,8 years:4.07%,9 years:3.51%
home_ownership,object,2260701,0.00%,6,MORTGAGE:49.16%,RENT:39.59%,OWN:11.19%,ANY:0.04%,OTHER:0.01%,RENT:39.59%,OWN:11.19%,ANY:0.04%,OTHER:0.01%,NONE:0.00%
verification_status,object,2260701,0.00%,3,Source Verified:39.20%,Not Verified:32.95%,Verified:27.85%,None,None,None,None,Source Verified:39.20%,Not Verified:32.95%,Verified:27.85%
issue_d,object,2260701,0.00%,139,Mar-2016:2.74%,Oct-2015:2.15%,May-2018:2.05%,Oct-2018:2.05%,Aug-2018:2.04%,Aug-2007:0.00%,Jul-2007:0.00%,Sep-2008:0.00%,Sep-2007:0.00%,Jun-2007:0.00%
loan_status,object,2260701,0.00%,9,Fully Paid:47.63%,Current:38.85%,Charged Off:11.88%,Late (31-120 days):0.95%,In Grace Period:0.37%,In Grace Period:0.37%,Late (16-30 days):0.19%,Does not meet the credit policy. Status:Fully ...,Does not meet the credit policy. Status:Charge...,Default:0.00%


##### int和float型变量是比较好处理的。object里有字符、日期、nan，需要丢弃和转换一些变量。

In [9]:
# 要丢弃的列：对预测没有用的列（比如url）；高缺失值（>90%）的列；高不平衡变量；冗余变量（邮编之于州代码）；以及unique值过多的object列（例如emp_title），这类列很难做分箱
# 按缺失率丢失可以用toad，但太懒人了

df1 = data.drop(columns=var_glance[var_glance['missing'].apply(lambda x: float(x.strip('%'))/100.0)>0.9].index.tolist())
df2 = df1.drop(columns=['id', 'url']) # 丢弃无用列
df3 = df2.drop(columns=['zip_code', 'title', 'loan_status']) # 丢弃冗余列
df4 = df3.drop(columns=['pymnt_plan', 'hardship_flag']) # 丢弃高不平衡列（99%）
data_nodrop = df4.drop(columns=['emp_title']) # 丢弃高离散列

In [10]:
data_nodrop.shape

(2260701, 106)

In [11]:
# 日期转换的列：issue_d, earliest_cr_line, last_pymnt_d, next_pymnt_d, last_credit_pull_d
# 注意：last_pymnt_d, next_pymnt_d 存在缺失值，填充的逻辑需要注意

def date_to_duration(df, columns):
    df2 = df
    present = pd.to_datetime('2021-12-31')  # 这个数据集只到2018Q4，所以假设现在是2021Q4最后一天
    for items in columns:
        #print(items)
        df2[items] = pd.to_datetime(df2[items], errors='ignore', format='%b-%Y')
        col_max =  df2[items].max()
        df2[items] = df2[items].apply(lambda x: col_max if (pd.isna(x)==True) else x)  # 处理所有nan和NaT值
        #print(sum(pd.isna(df2[items])))
        df2['mths_since_' + items] = df2[items].apply(lambda x: round(pd.to_numeric((present - x)/np.timedelta64(1, 'M'))))
    df2.drop(columns=columns, inplace=True)
    return(df2)

In [12]:
data_nodrop_dateconvert = date_to_duration(data_nodrop, ['issue_d','earliest_cr_line','last_pymnt_d','next_pymnt_d','last_credit_pull_d'])

In [13]:
# 清洗整个数据集的nan。之所以要放在日期清洗后，是因为日期有自己的填充逻辑
# 其它变量的填充应当按照众数，还是中位数？——如果按众数，是否会减弱该变量对坏客户的预测能力（target=1的样本在某变量上取值和target=0是一样的，这合适吗）
# 但一般来说，中位数和众数策略差不多。众数策略特别的优势在于，对字符变量也能适用。

from sklearn.impute import SimpleImputer

def optional_imputer(df, strategy):
    imp_mf = SimpleImputer(missing_values=np.nan, strategy=strategy)
    impdata = imp_mf.fit_transform(df)
    impdf = pd.DataFrame(impdata, columns=df.columns)
    df2 = impdf.astype({'target': 'int64'}, copy=True)
    return(df2)

In [14]:
data_nodrop_dateconvert_mfnan = optional_imputer(data_nodrop_dateconvert, 'most_frequent')

In [15]:
# 具有排序意义的列：grade, sub_grade, emp_length, term
# 使用map映射为数值，或用labelencoder更便捷

from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
le_cols = ['grade', 'sub_grade', 'emp_length', 'term', 'debt_settlement_flag']
for item in le_cols:
    data_nodrop_dateconvert_mfnan[item] = le.fit_transform(data_nodrop_dateconvert_mfnan[item])

In [16]:
# 不具有排序意义的列：home_ownership, verification_status, purpose，addr_state，initial_list_status，application_type，disbursement_method
# 使用get_dummies方法，令其取值为稀疏矩阵，可以用于建模。但若取值太过离散，矩阵维度过高，该方法就不好用。

dummy_cols = ['home_ownership', 'verification_status', 'purpose', 'addr_state', 
'initial_list_status', 'application_type', 'disbursement_method']
dummy_df = pd.get_dummies(data_nodrop_dateconvert_mfnan[dummy_cols])
data_ndm = pd.concat([data_nodrop_dateconvert_mfnan, dummy_df], axis=1)
data_ndm.drop(columns=dummy_cols, inplace=True)

In [17]:
data_ndm.shape

(2260701, 179)

In [18]:
data_ndm.to_feather(path + 'cleaned_accepted_2007_to_2018Q4.feather')

In [19]:
data_ndm.to_parquet(path + 'cleaned_accepted_2007_to_2018Q4.parquet', engine='fastparquet')